In [1]:
%%bash

# checking if kaggle API is configured

if [ ! -d ~/.kaggle/ ]; then

    echo "Kaggle credentials are not configured"

else

    echo "Kaggle API is already configured"

fi

Kaggle API is already configured


In [9]:
%%bash

# checking if data is downloaded

if [ ! -d data/ ]; then

  echo "Downloading dataset..."
  kaggle competitions download nlp-getting-started

  echo "Unzipping datasets"
  unzip -qq nlp-getting-started.zip -d data/
  
  rm -rf nlp-getting-started.zip

else

  echo "Dataset already dataset downloaded."

fi

Dataset already dataset downloaded.


In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [11]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [12]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [13]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [14]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [15]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [16]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

In [17]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

In [18]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.60355649, 0.57484457, 0.64485082])

In [19]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [20]:
sample_submission = pd.read_csv("data/sample_submission.csv")

In [21]:
sample_submission["target"] = clf.predict(test_vectors)

In [22]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [23]:
sample_submission.to_csv("submission.csv", index=False)

In [24]:
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "testing API submission"

100%|██████████████████████████████████████| 22.2k/22.2k [00:03<00:00, 6.54kB/s]
Successfully submitted to Real or Not? NLP with Disaster Tweets